<a href="https://colab.research.google.com/github/teticio/aventuras-con-textos/blob/master/BERT_entiende.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicir las palabras que faltan en una frase con BERT

El entrenamiento de los modelos de BERT pre-entrenados consiste en realizar dos tareas no supervisadas: (1) adivinar las palabras que faltan en una frase y (2) determinar si una frase sigue la otra o no. En este notebook, vamos a poner un modelo pre-entrenado a prueba con la primera tarea.

Se puede adaptar para generar frases enteras de texto "a boleo". Ver [Bert Babble](https://colab.research.google.com/drive/1MxKZGtQ9SSBjTK5ArsZ5LKhkztzg52RV).

# Predict the missing words in a sentence with BERT

The training of pre-trained BERT models consists of performing two unsupervised tasks: (1) guess the missing words in a sentence and (2) determine if one sentence follows the other or not. In this notebook, we will put a pre-trained model to the test with the first task.

It can be adapted to generate whole sentences of random text. See [Bert Babble](https://colab.research.google.com/drive/1MxKZGtQ9SSBjTK5ArsZ5LKhkztzg52RV).

### Descargar el modelo pre-entrenado de BERT

### Download the pre-trained BERT model

In [1]:
from keras.utils import get_file

# https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
# https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip

url = 'https://storage.googleapis.com/bert_models/2019_05_30/'  #@param {type : 'string'}
modelo = 'wwm_uncased_L-24_H-1024_A-16'  #@param {type : 'string'}
get_file(modelo + '.zip',
         origin=url + modelo + '.zip',
         extract=True,
         archive_format="zip",
         cache_dir='./',
         cache_subdir='')

Using TensorFlow backend.
/usr/lib/python3/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/lib/python3/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/lib/python3/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/lib/python3/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type

'./wwm_uncased_L-24_H-1024_A-16.zip'

### Instalar e importar BERT

### Install and import BERT

In [2]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
    sys.path += ['bert_repo']

# import python modules defined by BERT
import modeling as tfm
import tokenization as tft
import run_squad as rs

W1001 21:58:28.022922 140207269025600 deprecation_wrapper.py:119] From bert_repo/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



### Importar las librerías

### Import libraries

In [3]:
import tensorflow as tf
from keras.preprocessing import sequence
import numpy as np

model_dir = './' + modelo + '/'
vocab_file = model_dir + "vocab.txt"
bert_config_file = model_dir + "bert_config.json"
init_checkpoint = model_dir + "bert_model.ckpt"
max_seq_length = 512
max_predictions_per_seq = 20

### Configurar BERT y preparar el tokenizador

### Configure BERT and prepare the tokenizer

In [4]:
# convert to lower case?
convertir_a_minusculas = 'uncased' in modelo
bert_config = tfm.BertConfig.from_json_file(bert_config_file)
tokenizer = tft.FullTokenizer(vocab_file=vocab_file,
                              do_lower_case=convertir_a_minusculas)

W1001 21:58:28.036324 140207269025600 deprecation_wrapper.py:119] From bert_repo/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



### Definir los inputs al modelo

### Define the inputs to the model

In [5]:
input_ids = tf.placeholder(name='input_ids',
                           shape=(1, max_seq_length),
                           dtype='int32')
input_mask = tf.placeholder(name='input_mask',
                            shape=(1, max_seq_length),
                            dtype='int32')
segment_ids = tf.placeholder(name='segment_ids',
                             shape=(1, max_seq_length),
                             dtype='int32')

### Construir el modelo

In [6]:
(start_logits, end_logits) = rs.create_model(bert_config=bert_config,
                                             is_training=False,
                                             input_ids=input_ids,
                                             input_mask=input_mask,
                                             segment_ids=segment_ids,
                                             use_one_hot_embeddings=False)

W1001 21:58:28.113921 140207269025600 deprecation_wrapper.py:119] From bert_repo/modeling.py:171: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1001 21:58:28.117491 140207269025600 deprecation_wrapper.py:119] From bert_repo/modeling.py:409: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W1001 21:58:28.132351 140207269025600 deprecation_wrapper.py:119] From bert_repo/modeling.py:490: The name tf.assert_less_equal is deprecated. Please use tf.compat.v1.assert_less_equal instead.

W1001 21:58:28.603485 140207269025600 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1001 21:

### Build the model

### Inizializar los pesos con el checkpoint del modelo pre-entrenado

### Initialize the weights with the checkpoint of the pre-trained model

In [7]:
(assignment_map,
 initialized_variable_names) = tfm.get_assignment_map_from_checkpoint(
     tf.trainable_variables(), init_checkpoint)
tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

### Crear los inputs

### Create the inputs

In [8]:
# context
contexto = "In meteorology, precipitation is any product of the condensation of atmospheric water vapor that falls under gravity. The main forms of precipitation include drizzle, rain, sleet, snow, graupel and hail… Precipitation forms as smaller droplets coalesce via collision with other rain drops or ice crystals within a cloud. Short, intense periods of rain in scattered locations are called “showers”."  #@param {type : "string"}
# question
pregunta = "Where do water droplets collide with ice crystals to form precipitation?"  #@param
tokens = ['[CLS]'] + tokenizer.tokenize(pregunta) + ['[SEP]'] + tokenizer.tokenize(contexto) + ['[SEP]']
input_ids_ = tokenizer.convert_tokens_to_ids(tokens)
len_seg = tokens.index('[SEP]') + 1
segment_ids_ = [0] * len_seg + [1] * (len(input_ids_) - len_seg)
input_mask_ = [1] * len(input_ids_)

In [9]:
len(input_ids_)

97

### Hacer que las secuencias tengan el mismo tamaño

### Make sequences the same size

In [10]:
input_ids_ = sequence.pad_sequences([input_ids_],
                                    maxlen=max_seq_length,
                                    padding='post',
                                    value=0)
segment_ids_ = sequence.pad_sequences([segment_ids_],
                                      maxlen=max_seq_length,
                                      padding='post',
                                      value=0)
input_mask_ = sequence.pad_sequences([input_mask_],
                                     maxlen=max_seq_length,
                                     padding='post',
                                     value=0)

### Invocar el modelo

### Invoke the model

In [11]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tup = sess.run(
        (start_logits, end_logits),
        feed_dict={
            input_ids: input_ids_,
            input_mask: input_mask_,
            segment_ids: segment_ids_,
        })

In [12]:
start = np.argmax(tup[0])
end = np.argmax(tup[1])

In [13]:
start, end

(64, 4)

### Convertir los tokens a texto (las predicciones están en mayúsculas)

### Convert tokens to text (predictions are in uppercase)

In [14]:
text = ''
leave_space = False
for i, token in enumerate(tokens):
    if i == start:
        text += "***"
    if i == end:
        text += "+++"
    if token == '[CLS]' or token == '[SEP]':
        continue
    if token[0:2] == '##':
        text += token[2:]
    else:
        if leave_space and token[0].isalnum():
            text += ' '
        text += token
    leave_space = token[-1] != "'" and token[-1] != "-"
print(text)

where do water+++ droplets collide with ice crystals to form precipitation? in meteorology, precipitation is any product of the condensation of atmospheric water vapor that falls under gravity. the main forms of precipitation include drizzle, rain, sleet, snow, graupel and hail… precipitation forms as smaller*** droplets coalesce via collision with other rain drops or ice crystals within a cloud. short, intense periods of rain in scattered locations are called“ showers”.
